In [3]:
import torch
from torch_geometric.data import Data
from torch.utils.data import Dataset
import json
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
from utils import flattenStructure

In [4]:
from data import *
dataset = semeval2017Dataset(dataPath='../dataset/semeval2017-task8/', 
                             type='train',
                             w2vPath='../dataset/glove/',
                             w2vDim=25)

In [5]:
from torch.utils.data import DataLoader
loader = DataLoader(dataset, shuffle=True)

In [7]:
from ABGCN import *
from torch import optim
model = ABGCN(w2vDim = 25,
              s2vDim = 128,
              gcnHiddenDim = 128,
              rumorFeatureDim = 128,
              numRumorTag = 3,
              numStanceTag = 4,
              s2vMethon = 'l')
device = torch.device('cuda')
model = model.set_device(device)
loss_func = torch.nn.CrossEntropyLoss(reduction='mean').to(device)
optimizer = optim.Adagrad(model.parameters(), lr=1e-3, weight_decay=1e-2)
        

In [10]:
from tqdm import tqdm

totalLoss = 0.
curNum = 1
testdata = dataset[0]
for data in tqdm(iter(loader), 
                 desc="[epoch {:d}, rumor]".format(1),
                 leave=False, 
                 ncols=100):
    # 抹除dataloader生成batch时对数据的升维
    data['threadId'] = data['threadId'][0]
    data['threadIndex'] = data['threadIndex'][0]
    for i in range(len(data['nodeFeature'])):
        data['nodeFeature'][i] = data['nodeFeature'][i].view((data['nodeFeature'][i].shape[1], data['nodeFeature'][i].shape[2]))
    data['edgeIndexTD'] = data['edgeIndexTD'].view((data['edgeIndexTD'].shape[1], data['edgeIndexTD'].shape[2]))
    data['edgeIndexBU'] = data['edgeIndexBU'].view((data['edgeIndexBU'].shape[1], data['edgeIndexBU'].shape[2]))
    data['rumorTag'] = data['rumorTag'].view((data['rumorTag'].shape[1]))
    data['stanveTag'] = data['stanveTag'].view((data['stanveTag'].shape[1]))
    rumorTag = data['rumorTag'].to(device)

    optimizer.zero_grad()
    p = model.forwardRumor(data)
    loss = loss_func(p, rumorTag)
    totalLoss += loss
    loss.backward()
    optimizer.step()
    curNum += 1
